In [1]:
# pip install -U albumentations
# Đây là code Recognize với CNN + Transformer Decoder

In [2]:
import os
import cv2
import math
import numpy as np
import random
import time
from pathlib import Path
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, GradScaler
import torchvision.transforms as transforms
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from torch.nn import TransformerDecoder, TransformerDecoderLayer
import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm.auto import tqdm

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(42)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Đang sử dụng thiết bị: {device}")

Đang sử dụng thiết bị: cuda


/usr/local/lib/python3.11/dist-packages/albumentations/__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.6' (you have '2.0.4'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [ ]:
#TFMdecoder file
class SimpleTokenizer:
    def __init__(self):
        self.special_tokens = ["<PAD>", "<SOS>", "<EOS>", "<UNK>"]
        self.pad_token_idx = 0  # <PAD>
        self.sos_token_idx = 1  # <SOS>
        self.eos_token_idx = 2  # <EOS>
        self.unk_token_idx = 3  # <UNK>
        
        chars = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789"
        chars += "àáảãạăắằẳẵặâấầẩẫậèéẻẽẹêếềểễệìíỉĩịòóỏõọôốồổỗộơớờởỡợùúủũụưứừửữựỳýỷỹỵđ"
        chars += "ÀÁẢÃẠĂẮẰẲẴẶÂẤẦẨẪẬÈÉẺẼẸÊẾỀỂỄỆÌÍỈĨỊÒÓỎÕỌÔỐỒỔỖỘƠỚỜỞỠỢÙÚỦŨỤƯỨỪỬỮỰỲÝỶỸỴĐ"
        chars += " .,;:!?()-/\\\"'[]{}@#$%^&*+=_<>|~`₫$€¥£¢°×÷√≤≥±≠∞≈"
        
        self.char_to_idx = {char: idx + len(self.special_tokens) for idx, char in enumerate(chars)}
        self.idx_to_char = {idx + len(self.special_tokens): char for idx, char in enumerate(chars)}
        
        for idx, token in enumerate(self.special_tokens):
            self.idx_to_char[idx] = token
            self.char_to_idx[token] = idx
        
        # Kích thước từ điển
        self.vocab_size = len(self.char_to_idx)
        
        print(f"Kích thước từ điển: {self.vocab_size}")
    
    def encode(self, text, max_length=100, add_special_tokens=True):
        indices = []
        
        if add_special_tokens:
            indices.append(self.sos_token_idx)
        
        for char in text:
            if char in self.char_to_idx:
                indices.append(self.char_to_idx[char])
            else:
                indices.append(self.unk_token_idx)
        
        if add_special_tokens:
            indices.append(self.eos_token_idx)
        
        # Padding
        if len(indices) < max_length:
            indices += [self.pad_token_idx] * (max_length - len(indices))
        else:
            indices = indices[:max_length]
            if add_special_tokens:
                indices[-1] = self.eos_token_idx
        
        return indices
    
    def decode(self, indices, remove_special_tokens=True):
        text = ""
        
        for idx in indices:
            if remove_special_tokens and idx in [self.pad_token_idx, self.sos_token_idx, self.eos_token_idx]:
                continue
            
            if idx in self.idx_to_char:
                text += self.idx_to_char[idx]
            else:
                text += self.idx_to_char[self.unk_token_idx]
        
        return text

def preprocess_image(image, adaptive_threshold=False):
    if len(image.shape) == 3 and image.shape[2] == 3:
        gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    else:
        gray = image
    
    blurred = cv2.GaussianBlur(gray, (3, 3), 0)
    
    if adaptive_threshold:
        binary = cv2.adaptiveThreshold(
            blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
            cv2.THRESH_BINARY, 11, 2
        )
        result = cv2.cvtColor(binary, cv2.COLOR_GRAY2RGB)
    else:
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        equalized = clahe.apply(blurred)
        
        result = cv2.cvtColor(equalized, cv2.COLOR_GRAY2RGB)
    
    return result

In [ ]:
class VietnameseOCRDataset(Dataset):
    def __init__(self, images_dir, annotation_file, tokenizer, transform=None, 
                 preprocess=True, adaptive_threshold=False, max_samples=None,
                 max_target_length=100):
        self.images_dir = images_dir
        self.transform = transform
        self.tokenizer = tokenizer
        self.preprocess = preprocess
        self.adaptive_threshold = adaptive_threshold
        self.max_target_length = max_target_length
        
        with open(annotation_file, 'r', encoding='utf-8') as f:
            lines = f.readlines()
        
        self.samples = []
        skipped = 0
        
        for line in lines:
            parts = line.strip().split('\t')
            if len(parts) == 2:
                image_name, text = parts
                image_path = os.path.join(images_dir, image_name)
                if os.path.exists(image_path):
                    if len(text.strip()) > 0:
                        self.samples.append((image_path, text))
                        if max_samples and len(self.samples) >= max_samples:
                            break
                else:
                    skipped += 1
        
        print(f"Đã tải {len(self.samples)} mẫu từ {annotation_file}")
        if skipped > 0:
            print(f"Đã bỏ qua {skipped} mẫu do không tìm thấy file ảnh")
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        image_path, text = self.samples[idx]
        
        image = cv2.imread(image_path)
        if image is None:
            print(f"Lỗi đọc ảnh: {image_path}")
            image = np.zeros((32, 100, 3), dtype=np.uint8)
        else:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
            if self.preprocess:
                image = preprocess_image(image, self.adaptive_threshold)
        
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']
        
        target = torch.tensor(self.tokenizer.encode(text, max_length=self.max_target_length), dtype=torch.long)
        
        target_input = target[:-1]  # Bỏ EOS token ở cuối
        target_output = target[1:]  # Bỏ SOS token ở đầu
        
        return image, target_input, target_output, text

def get_ocr_transforms(height=32, width=320, is_train=True):
    if is_train:
        return A.Compose([
            A.Resize(height=height, width=width),
            
            A.OneOf([
                A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.3, p=0.7),
                A.CLAHE(clip_limit=4.0, p=0.3),
            ], p=0.5),
            
            A.ShiftScaleRotate(shift_limit=0.02, scale_limit=0.1, rotate_limit=1, p=0.3),
            
            A.OneOf([
                A.GaussNoise(p=0.3),
                A.GaussianBlur(blur_limit=3, p=0.3),
            ], p=0.2),
            
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ToTensorV2()
        ])
    else:
        return A.Compose([
            A.Resize(height=height, width=width),
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ToTensorV2()
        ])

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:, :x.size(1)]
        return self.dropout(x)

class CNNTransformerDecoder(nn.Module):
    def __init__(self, vocab_size, input_channels=3, hidden_dim=256, nhead=4, 
                 num_encoder_layers=3, num_decoder_layers=3, dim_feedforward=1024, dropout=0.1):
        super(CNNTransformerDecoder, self).__init__()
        
        self.cnn = nn.Sequential(
            nn.Conv2d(input_channels, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=(2, 1), stride=(2, 1)),
            
            nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1), 
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=(2, 1), stride=(2, 1)),
            
            nn.Conv2d(512, hidden_dim, kernel_size=2, stride=1, padding=0),
            nn.BatchNorm2d(hidden_dim),
            nn.ReLU(inplace=True)
        )
        
        encoder_layers = TransformerEncoderLayer(
            d_model=hidden_dim, 
            nhead=nhead, 
            dim_feedforward=dim_feedforward, 
            dropout=dropout,
            batch_first=True
        )
        self.transformer_encoder = TransformerEncoder(encoder_layers, num_encoder_layers)
        
        self.token_embedding = nn.Embedding(vocab_size, hidden_dim)
        self.pos_encoder = PositionalEncoding(hidden_dim, dropout)
        
        decoder_layers = TransformerDecoderLayer(
            d_model=hidden_dim, 
            nhead=nhead, 
            dim_feedforward=dim_feedforward, 
            dropout=dropout,
            batch_first=True
        )
        self.transformer_decoder = TransformerDecoder(decoder_layers, num_decoder_layers)
        
        # Output Projection
        self.fc_out = nn.Linear(hidden_dim, vocab_size)
        
        self._init_weights()
        
        # Lưu lại kích thước từ điển
        self.vocab_size = vocab_size
    
    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
    
    def create_square_subsequent_mask(self, sz, device):
        mask = (torch.triu(torch.ones(sz, sz, device=device)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask
    
    def create_padding_mask(self, src, pad_idx):
        return (src == pad_idx).float().unsqueeze(-1)
    
    def encode(self, src):
        # Trích xuất đặc trưng với CNN
        features = self.cnn(src)  # [B, C, H, W]
        
        # Đưa về dạng sequence
        features = features.squeeze(2)  # [B, C, W]
        features = features.permute(0, 2, 1)  # [B, W, C]
        
        # Qua Transformer Encoder
        memory = self.transformer_encoder(features)
        
        return memory
    
    def decode(self, tgt, memory, tgt_mask=None, tgt_padding_mask=None):
        # Embedding cho target
        tgt_embedded = self.token_embedding(tgt)
        tgt_embedded = self.pos_encoder(tgt_embedded)
        
        # Transformer Decoder
        output = self.transformer_decoder(
            tgt_embedded, memory, 
            tgt_mask=tgt_mask,
            tgt_key_padding_mask=tgt_padding_mask
        )
        
        # Projection để lấy logits
        output = self.fc_out(output)
        
        return output
    
    def forward(self, src, tgt, tgt_mask=None, pad_idx=0):
        # Encode
        memory = self.encode(src)
        
        # Tạo tgt_mask nếu chưa có
        if tgt_mask is None and tgt is not None:
            tgt_mask = self.create_square_subsequent_mask(tgt.size(1), tgt.device)
        
        # Tạo padding mask cho tgt
        tgt_padding_mask = (tgt == pad_idx).to(tgt.device) if tgt is not None else None
        
        # Decode
        output = self.decode(tgt, memory, tgt_mask, tgt_padding_mask)
        
        return output
    
    def generate(self, src, tokenizer, max_length=100, beam_size=1):
        self.eval()
        
        with torch.no_grad():
            memory = self.encode(src)
            
            sos_token = tokenizer.sos_token_idx
            eos_token = tokenizer.eos_token_idx
            
            if beam_size == 1:
                dec_input = torch.LongTensor([[sos_token]]).to(src.device)
                
                for i in range(max_length):
                    tgt_mask = self.create_square_subsequent_mask(dec_input.size(1), dec_input.device)
                    
                    out = self.decode(dec_input, memory, tgt_mask)
                    
                    prob = F.softmax(out[:, -1], dim=-1)
                    _, next_token = torch.max(prob, dim=1)
                    next_token = next_token.unsqueeze(1)
                    
                    dec_input = torch.cat([dec_input, next_token], dim=1)
                    
                    if next_token.item() == eos_token:
                        break
                
                predicted_indices = dec_input.squeeze().cpu().numpy().tolist()
                predicted_text = tokenizer.decode(predicted_indices)
                
                return predicted_text
            else:
                pass

In [ ]:
def collate_fn(batch):
    """Hàm custom collate_fn để xử lý các chuỗi có độ dài khác nhau"""
    images, target_inputs, target_outputs, texts = zip(*batch)
    
    heights = [img.shape[1] for img in images]
    widths = [img.shape[2] for img in images]
    
    if len(set(heights)) > 1 or len(set(widths)) > 1:
        print(f"Warning: Không đồng nhất kích thước trong batch: heights={heights}, widths={widths}")
        
        target_height = max(set(heights), key=heights.count)
        target_width = max(set(widths), key=widths.count)
        
        resized_images = []
        for img in images:
            if img.shape[1] != target_height or img.shape[2] != target_width:
                resized = F.interpolate(img.unsqueeze(0), size=(target_height, target_width), 
                                        mode='bilinear', align_corners=False).squeeze(0)
                resized_images.append(resized)
            else:
                resized_images.append(img)
        
        images = resized_images
    
    images = torch.stack(images)
    target_inputs = torch.stack(target_inputs)
    target_outputs = torch.stack(target_outputs)
    
    return images, target_inputs, target_outputs, texts

def levenshtein_distance(s1, s2):
    """Tính khoảng cách Levenshtein giữa hai chuỗi"""
    if len(s1) < len(s2):
        return levenshtein_distance(s2, s1)
    
    if len(s2) == 0:
        return len(s1)
    
    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1
            deletions = current_row[j] + 1
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row
    
    return previous_row[-1]

def evaluate(model, dataloader, tokenizer, device, max_length=100, max_samples=None):
    """Đánh giá mô hình trên tập dữ liệu"""
    model.eval()
    
    all_predictions = []
    all_targets = []
    
    print("Đang đánh giá... ", end="", flush=True)
    
    with torch.no_grad():
        for idx, (images, _, _, texts) in enumerate(dataloader):
            if idx % 5 == 0:
                print(".", end="", flush=True)
                
            images = images.to(device)
            
            predictions = []
            for i in range(images.size(0)):
                pred_text = model.generate(images[i:i+1], tokenizer, max_length=max_length)
                predictions.append(pred_text)
            
            all_predictions.extend(predictions)
            all_targets.extend(texts)
            
            if max_samples and len(all_predictions) >= max_samples:
                all_predictions = all_predictions[:max_samples]
                all_targets = all_targets[:max_samples]
                break
    
    print(" Hoàn thành!")
    print(f"Đã đánh giá {len(all_predictions)} mẫu")
    
    correct = sum(p == t for p, t in zip(all_predictions, all_targets))
    accuracy = correct / len(all_predictions) if all_predictions else 0
    
    total_cer = 0
    for pred, target in zip(all_predictions, all_targets):
        distance = levenshtein_distance(pred, target)
        target_len = max(len(target), 1)  # Tránh chia cho 0
        total_cer += distance / target_len
    avg_cer = total_cer / len(all_predictions) if all_predictions else 1
    
    return accuracy, avg_cer, all_predictions, all_targets

In [ ]:
def train_cnn_transformer_decoder(model, train_loader, val_loader, tokenizer, output_dir='output', 
                 epochs=30, lr=0.0005, weight_decay=1e-5, device='cuda', patience=10):
    """Hàm huấn luyện mô hình CNN + Transformer Decoder"""
    os.makedirs(output_dir, exist_ok=True)
    
    criterion = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_idx)
    
    optimizer = torch.optim.AdamW(
        model.parameters(), 
        lr=lr, 
        betas=(0.9, 0.999),
        weight_decay=weight_decay
    )
    
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, 
        mode='min', 
        factor=0.5,
        patience=3,
        min_lr=1e-6,
        verbose=True
    )
    
    if hasattr(torch.amp, 'GradScaler'):
        scaler = torch.amp.GradScaler('cuda' if device == 'cuda' else 'cpu')
    else:
        scaler = GradScaler(enabled=(device == 'cuda'))
    
    history = {
        'train_loss': [],
        'val_loss': [],
        'accuracy': [],
        'cer': [],
        'learning_rate': []
    }
    
    best_cer = float('inf')
    no_improve_epochs = 0
    best_epoch = -1
    
    accumulation_steps = 2
    
    for epoch in range(epochs):
        print(f"\n{'='*20} Epoch {epoch+1}/{epochs} {'='*20}")
        start_time = time.time()
        
        model.train()
        train_loss = 0.0
        batch_count = 0
        optimizer.zero_grad()
        
        progress_bar = tqdm(train_loader, desc=f"Training")
        for i, (images, target_inputs, target_outputs, _) in enumerate(progress_bar):
            images = images.to(device)
            target_inputs = target_inputs.to(device)
            target_outputs = target_outputs.to(device)
            
            if hasattr(torch.amp, 'autocast'):
                ctx_manager = torch.amp.autocast('cuda' if device == 'cuda' else 'cpu')
            else:
                ctx_manager = autocast(enabled=(device == 'cuda'))
            
            with ctx_manager:
                tgt_mask = model.create_square_subsequent_mask(target_inputs.size(1), device)
                
                output = model(images, target_inputs, tgt_mask, tokenizer.pad_token_idx)
                
                output_flat = output.reshape(-1, output.size(-1))
                target_flat = target_outputs.reshape(-1)
                
                loss = criterion(output_flat, target_flat)
                loss = loss / accumulation_steps
            
            if device == 'cuda':
                scaler.scale(loss).backward()
                
                if (i + 1) % accumulation_steps == 0 or (i + 1 == len(train_loader)):
                    scaler.unscale_(optimizer)
                    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                    scaler.step(optimizer)
                    scaler.update()
                    optimizer.zero_grad()
            else:
                loss.backward()
                
                if (i + 1) % accumulation_steps == 0 or (i + 1 == len(train_loader)):
                    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                    optimizer.step()
                    optimizer.zero_grad()
            
            current_lr = optimizer.param_groups[0]['lr']
            train_loss += loss.item() * accumulation_steps
            batch_count += 1
            progress_bar.set_postfix(loss=f"{loss.item() * accumulation_steps:.4f}", lr=f"{current_lr:.7f}")
        
        avg_train_loss = train_loss / batch_count if batch_count > 0 else float('inf')
        history['train_loss'].append(avg_train_loss)
        history['learning_rate'].append(optimizer.param_groups[0]['lr'])
        
        model.eval()
        val_loss = 0.0
        val_batch_count = 0
        
        with torch.no_grad():
            for images, target_inputs, target_outputs, _ in tqdm(val_loader, desc="Validation"):
                images = images.to(device)
                target_inputs = target_inputs.to(device)
                target_outputs = target_outputs.to(device)
                
                tgt_mask = model.create_square_subsequent_mask(target_inputs.size(1), device)
                
                output = model(images, target_inputs, tgt_mask, tokenizer.pad_token_idx)
                
                output_flat = output.reshape(-1, output.size(-1))
                target_flat = target_outputs.reshape(-1)
                
                loss = criterion(output_flat, target_flat)
                val_loss += loss.item()
                val_batch_count += 1
        
        avg_val_loss = val_loss / val_batch_count if val_batch_count > 0 else float('inf')
        history['val_loss'].append(avg_val_loss)
        
        scheduler.step(avg_val_loss)
        
        accuracy, cer, predictions, targets = evaluate(
            model, val_loader, tokenizer, device, max_samples=100
        )
        
        history['accuracy'].append(accuracy)
        history['cer'].append(cer)
        
        epoch_time = time.time() - start_time
        print(f"\nEpoch {epoch+1}/{epochs} - Time: {epoch_time:.1f}s")
        print(f"  Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")
        print(f"  Accuracy: {accuracy:.4f}, CER: {cer:.4f}")
        print(f"  Learning Rate: {optimizer.param_groups[0]['lr']:.7f}")
        
        print("\nMẫu dự đoán:")
        for i in range(min(5, len(predictions))):
            print(f"  Mẫu {i+1}:")
            print(f"  Dự đoán: '{predictions[i]}'")
            print(f"  Thực tế: '{targets[i]}'")
            print()
        
        if cer < best_cer:
            best_cer = cer
            best_epoch = epoch + 1
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'scheduler_state_dict': scheduler.state_dict(),
                'train_loss': avg_train_loss,
                'val_loss': avg_val_loss,
                'accuracy': accuracy,
                'cer': cer,
                'history': history,
            }, os.path.join(output_dir, 'best_model.pth'))
            print(f"  Đã lưu mô hình tốt nhất với CER: {cer:.4f}!")
            no_improve_epochs = 0
        else:
            no_improve_epochs += 1
            print(f"  Không cải thiện CER. Còn {patience - no_improve_epochs} epochs nữa sẽ dừng early stopping.")
        
        # Lưu checkpoint
        if (epoch + 1) % 5 == 0 or epoch == epochs - 1:
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'scheduler_state_dict': scheduler.state_dict(),
                'train_loss': avg_train_loss,
                'val_loss': avg_val_loss,
                'accuracy': accuracy,
                'cer': cer,
                'history': history,
                'best_cer': best_cer,
                'best_epoch': best_epoch
            }, os.path.join(output_dir, f'checkpoint_epoch_{epoch+1}.pth'))
            print(f"  Lưu checkpoint tại epoch {epoch+1}")
        
        # Early stopping
        if no_improve_epochs >= patience:
            print(f"\nEarly stopping sau {patience} epochs không cải thiện.")
            break
    
    print(f"\nKết thúc huấn luyện sau {epoch+1} epochs")
    print(f"Mô hình tốt nhất tại epoch {best_epoch} với CER: {best_cer:.4f}")
    
    return model, history

In [ ]:
def predict_from_image_path(model, image_path, tokenizer, device, preprocess=True, max_length=100):
    """Dự đoán văn bản từ đường dẫn ảnh"""
    image = cv2.imread(image_path)
    if image is None:
        print(f"Không thể đọc ảnh từ {image_path}")
        return ""
    
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    if preprocess:
        image = preprocess_image(image)
    
    transform = get_ocr_transforms(is_train=False)
    image_tensor = transform(image=image)['image']
    
    model.eval()
    with torch.no_grad():
        image_tensor = image_tensor.unsqueeze(0).to(device)
        prediction = model.generate(image_tensor, tokenizer, max_length=max_length)
    
    return prediction


In [ ]:
def main():
    data_dir = '/kaggle/input/vietnamese-receipts-mc-ocr-2021'
    images_dir = os.path.join(data_dir, 'text_recognition_mcocr_data/text_recognition_mcocr_data')
    train_file = os.path.join(data_dir, 'text_recognition_train_data.txt')
    val_file = os.path.join(data_dir, 'text_recognition_val_data.txt')
    
    output_dir = 'output'
    os.makedirs(output_dir, exist_ok=True)
    
    if not os.path.exists(images_dir):
        print(f"Thư mục ảnh không tồn tại: {images_dir}")
        return
    if not os.path.exists(train_file):
        print(f"File train không tồn tại: {train_file}")
        return
    if not os.path.exists(val_file):
        print(f"File validation không tồn tại: {val_file}")
        return
    
    tokenizer = SimpleTokenizer()
    
    train_dataset = VietnameseOCRDataset(
        images_dir=images_dir,
        annotation_file=train_file,
        tokenizer=tokenizer,
        transform=get_ocr_transforms(height=32, width=320, is_train=True),
        preprocess=True,
        adaptive_threshold=False,
        max_target_length=100
    )
    
    val_dataset = VietnameseOCRDataset(
        images_dir=images_dir,
        annotation_file=val_file,
        tokenizer=tokenizer,
        transform=get_ocr_transforms(height=32, width=320, is_train=False),
        preprocess=True,
        adaptive_threshold=False,
        max_target_length=100
    )
    
    train_loader = DataLoader(
        train_dataset,
        batch_size=16,
        shuffle=True,
        num_workers=0,
        pin_memory=True if torch.cuda.is_available() else False,
        collate_fn=collate_fn
    )
    
    val_loader = DataLoader(
        val_dataset,
        batch_size=16,
        shuffle=False,
        num_workers=0,
        pin_memory=True if torch.cuda.is_available() else False,
        collate_fn=collate_fn
    )
    
    model = CNNTransformerDecoder(
        vocab_size=tokenizer.vocab_size,
        input_channels=3,
        hidden_dim=256,
        nhead=4,
        num_encoder_layers=3,
        num_decoder_layers=3,
        dim_feedforward=1024,
        dropout=0.1
    )
    
    model = model.to(device)
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"Tổng số tham số của mô hình: {total_params:,}")
    print(f"Số tham số có thể train: {trainable_params:,}")
    
    model, history = train_cnn_transformer_decoder(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        tokenizer=tokenizer,
        output_dir=output_dir,
        epochs=30,
        lr=0.0005,
        weight_decay=1e-5,
        device=device,
        patience=10
    )
    
    checkpoint = torch.load(os.path.join(output_dir, 'best_model.pth'))
    best_model = CNNTransformerDecoder(
        vocab_size=tokenizer.vocab_size,
        input_channels=3,
        hidden_dim=256,
        nhead=4,
        num_encoder_layers=3,
        num_decoder_layers=3,
        dim_feedforward=1024,
        dropout=0.1
    )
    
    best_model.load_state_dict(checkpoint['model_state_dict'])
    best_model = best_model.to(device)
    
    # Đánh giá mô hình tốt nhất
    accuracy, cer, predictions, targets = evaluate(
        best_model, val_loader, tokenizer, device, max_samples=200
    )
    
    print(f"\nĐánh giá mô hình tốt nhất:")
    print(f"Accuracy: {accuracy:.4f}, CER: {cer:.4f}")
    
    print("Quá trình huấn luyện và đánh giá hoàn tất!")

if __name__ == "__main__":
    main()

Kích thước từ điển: 248


/usr/local/lib/python3.11/dist-packages/albumentations/core/validation.py:58: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)


Đã tải 5285 mẫu từ /kaggle/input/vietnamese-receipts-mc-ocr-2021/text_recognition_train_data.txt
Đã tải 1300 mẫu từ /kaggle/input/vietnamese-receipts-mc-ocr-2021/text_recognition_val_data.txt
Tổng số tham số của mô hình: 10,686,200
Số tham số có thể train: 10,686,200

==================== Epoch 1/30 ====================


/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Training:   0%|          | 0/331 [00:00<?, ?it/s]

...

Training:   0%|          | 0/331 [00:00<?, ?it/s]

Validation:   0%|          | 0/82 [00:00<?, ?it/s]

Đang đánh giá... .. Hoàn thành!
Đã đánh giá 100 mẫu

Epoch 30/30 - Time: 55.3s
  Train Loss: 0.2814, Val Loss: 0.4578
  Accuracy: 0.3500, CER: 0.2771
  Learning Rate: 0.0005000

Mẫu dự đoán:
  Mẫu 1:
  Dự đoán: 'NHÀ SÁCH GD-TC CẨM PHẢ'
  Thực tế: 'NHÀ SÁCH GD-TC CẨM PHẢ'

  Mẫu 2:
  Dự đoán: 'ĐC: 212 Đường Trần Phú-Cẩm Phả'
  Thực tế: 'ĐC: 212 Đường Trần Phú-Cẩm Phả'

  Mẫu 3:
  Dự đoán: 'Thời gian: 08:42:43 - 14/08/2020'
  Thực tế: 'Thời gian 10:44:08-15/08/2020'

  Mẫu 4:
  Dự đoán: 'Tổng số thanh toán'
  Thực tế: 'Tổng số thanh toán:'

  Mẫu 5:
  Dự đoán: '10.000đ'
  Thực tế: '150.000đ'

  Đã lưu mô hình tốt nhất với CER: 0.2771!
  Lưu checkpoint tại epoch 30

Kết thúc huấn luyện sau 30 epochs
Mô hình tốt nhất tại epoch 30 với CER: 0.2771


/tmp/ipykernel_31/1251999806.py:96: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(os.path.join(output_dir, 'best_model.pth'))


Đang đánh giá... ... Hoàn thành!
Đã đánh giá 200 mẫu

Đánh giá mô hình tốt nhất:
Accuracy: 0.3400, CER: 0.3075
Quá trình huấn luyện và đánh giá hoàn tất!
